## Hybrid classifier - baseline + expert system

In [4]:
import torch
import joblib
from rocks_evaluation import SimpleCNN1D, UncertaintyAwareCNN1D, IntegratedRockClassifier, UncertaintyIntegratedRockClassifier, RockType, MineralGroups

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

baseline_model = SimpleCNN1D().to(device)  
# uncertainty_model = UncertaintyAwareCNN1D()

# For standard classification


# Model configuration and loading
best_repeat = 0
best_fold = 0
best_overall_accuracy = 0.9927
best_model_path = f'./models/model_repeat{best_repeat}_fold{best_fold}_acc{best_overall_accuracy:.4f}.pth'

# Initialize model and load weights
best_model = baseline_model.to(device)  # Make sure to import your model class
best_checkpoint = torch.load(best_model_path, weights_only=True)
best_model.load_state_dict(best_checkpoint['model_state_dict'])
best_model.eval()

# Load label encoder
label_encoder = joblib.load('mineral_label_encoder.joblib')

# Initialize rock analyzer
rock_analyzer = IntegratedRockClassifier(
    model=best_model,
    label_encoder=label_encoder,
    device=device,
)



FileNotFoundError: [Errno 2] No such file or directory: './models/model_repeat0_fold0_acc0.9927.pth'

## Rock Expert system
- Adapt into new inference rules

In [3]:
import torch
import numpy as np
import joblib
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from typing import Dict, List, Tuple
from collections import deque
from enum import Enum
from dataclasses import dataclass

class RockType(Enum):
    GRANITE = 'granite'
    LIMESTONE = 'limestone'
    SANDSTONE = 'sandstone'
    OTHER = 'other'

@dataclass
class MineralGroups:
    """Mineral groups definition following Γ notation"""
    def __init__(self):
        self.feldspars = {'albite', 'anorthite', 'orthoclase', 'sanidine'}  # Γ_f
        self.quartz = {'quartz'}  # Γ_q
        self.micas = {'annite', 'eastonite', 'margarite', 'muscovite', 'phlogopite'}  # Γ_m
        self.calcite = {'calcite'}  # Γ_c
        self.pyrite = {'pyrite'}  # Γ_p
        self.rutile = {'rutile'}  # Γ_r
        self.tourmaline = {'tourmaline'}  # Γ_t

class IntegratedRockClassifier:
    def __init__(self, model, label_encoder, device, window_size: int = 10):
        """
        Initialize integrated classifier that combines mineral classifier with rock type analysis
        
        Args:
            model: Trained PyTorch mineral classifier model
            label_encoder: Fitted LabelEncoder for mineral labels
            device: PyTorch device (cuda/cpu)
            window_size: Number of measurements to consider (fixed at 10 for new rules)
        """
        self.model = model
        self.label_encoder = label_encoder
        self.device = device
        self.window_size = 10  # Fixed at 10 per new rules
        self.mineral_groups = MineralGroups()
        self.measurements = []
        self.prediction_history = []
        self.ground_truth_history = []
        self.analysis_history = []
        
    def predict_mineral(self, spectrum: torch.Tensor) -> Tuple[str, np.ndarray]:
        """
        Predict mineral from spectrum using the trained model
        """
        self.model.eval()
        with torch.no_grad():
            if len(spectrum.shape) == 1:
                spectrum = spectrum.unsqueeze(0)
            spectrum = spectrum.to(self.device)
            
            outputs = self.model(spectrum)
            probabilities = torch.softmax(outputs, dim=1)
            
            probs = probabilities.cpu().numpy()[0]
            predicted_idx = np.argmax(probs)
            predicted_label = self.label_encoder.inverse_transform([predicted_idx])[0]
            
            return predicted_label, probs

    def check_accuracy_rule(self, predictions: List[str], ground_truth: List[str]) -> Dict:
        """
        Rule 1: Accuracy Rule
        ∑(mᵢ = tᵢ) ≥ 6, where i ∈ {1,...,10}
        """
        if len(predictions) != 10 or len(ground_truth) != 10:
            return {
                'satisfied': False,
                'accuracy': 0.0,
                'correct_predictions': 0
            }
            
        correct_predictions = sum(
            1 for pred, truth in zip(predictions, ground_truth)
            if pred.lower() == truth.lower()
        )
        accuracy = correct_predictions / 10
        
        return {
            'satisfied': accuracy >= 0.6,
            'accuracy': accuracy,
            'correct_predictions': correct_predictions
        }

    def check_mineral_assemblage_rules(self, predictions: List[str]) -> Dict:
        """
        Rule 2: Mineral Assemblage Rules for different rock types
        """
        predictions_lower = {pred.lower() for pred in predictions}
        
        # Check presence for each rock type
        granite_rules = {
            'feldspar_present': bool(predictions_lower & self.mineral_groups.feldspars),
            'quartz_present': bool(predictions_lower & self.mineral_groups.quartz),
            'mica_present': bool(predictions_lower & self.mineral_groups.micas)
        }
        
        limestone_rules = {
            'calcite_present': bool(predictions_lower & self.mineral_groups.calcite),
            'low_impurities': len(predictions_lower & (self.mineral_groups.quartz | 
                                                    self.mineral_groups.feldspars | 
                                                    self.mineral_groups.pyrite)) <= 2
        }
        
        sandstone_rules = {
            'quartz_present': bool(predictions_lower & self.mineral_groups.quartz),
            'feldspar_present': bool(predictions_lower & self.mineral_groups.feldspars),
            'accessory_present': bool(predictions_lower & (self.mineral_groups.micas | 
                                                        self.mineral_groups.rutile | 
                                                        self.mineral_groups.tourmaline))
        }
        
        # Get mineral counts
        mineral_counts = {
            'feldspars': sum(1 for p in predictions if p.lower() in self.mineral_groups.feldspars),
            'quartz': sum(1 for p in predictions if p.lower() in self.mineral_groups.quartz),
            'micas': sum(1 for p in predictions if p.lower() in self.mineral_groups.micas),
            'calcite': sum(1 for p in predictions if p.lower() in self.mineral_groups.calcite),
            'pyrite': sum(1 for p in predictions if p.lower() in self.mineral_groups.pyrite),
            'rutile': sum(1 for p in predictions if p.lower() in self.mineral_groups.rutile),
            'tourmaline': sum(1 for p in predictions if p.lower() in self.mineral_groups.tourmaline)
        }
        
        # Determine which rock type rules are satisfied
        is_granite = all(granite_rules.values())
        is_limestone = all(limestone_rules.values())
        is_sandstone = all(sandstone_rules.values())
        
        return {
            'satisfied': any([is_granite, is_limestone, is_sandstone]),
            'details': {
                'granite': granite_rules,
                'limestone': limestone_rules,
                'sandstone': sandstone_rules
            },
            'counts': mineral_counts,
            'rock_types': {
                'granite': is_granite,
                'limestone': is_limestone,
                'sandstone': is_sandstone
            }
        }

    def process_spectrum(self, spectrum: torch.Tensor, true_mineral: str = None) -> Dict:
        """
        Process a single spectrum measurement
        """
        # Get mineral prediction
        predicted_mineral, probabilities = self.predict_mineral(spectrum)
        
        # Store prediction and ground truth
        self.prediction_history.append(predicted_mineral)
        if true_mineral is not None:
            self.ground_truth_history.append(true_mineral)
        
        # Keep only the last 10 measurements
        if len(self.prediction_history) > 10:
            self.prediction_history.pop(0)
            if self.ground_truth_history:
                self.ground_truth_history.pop(0)
        
        # Perform rock analysis
        if len(self.prediction_history) == 10 and len(self.ground_truth_history) == 10:
            accuracy_analysis = self.check_accuracy_rule(
                self.prediction_history, 
                self.ground_truth_history
            )
            assemblage_analysis = self.check_mineral_assemblage_rules(
                self.prediction_history
            )
            
            # Determine final classification
            if accuracy_analysis['satisfied']:
                if assemblage_analysis['rock_types']['granite']:
                    classification = RockType.GRANITE
                elif assemblage_analysis['rock_types']['limestone']:
                    classification = RockType.LIMESTONE
                elif assemblage_analysis['rock_types']['sandstone']:
                    classification = RockType.SANDSTONE
                else:
                    classification = RockType.OTHER
            else:
                classification = RockType.OTHER
            
            rock_analysis = {
                'classification': classification.value,
                'accuracy_rule': accuracy_analysis,
                'assemblage_rules': assemblage_analysis,
            }
        else:
            rock_analysis = {
                'classification': RockType.OTHER.value,
                'status': 'Insufficient measurements',
                'current_count': len(self.prediction_history)
            }
        
        # Store analysis for plotting
        self.analysis_history.append({
            'mineral_prediction': predicted_mineral,
            'true_mineral': true_mineral,
            'rock_analysis': rock_analysis,
            'measurement_number': len(self.prediction_history)
        })
        
        return {
            'mineral_prediction': predicted_mineral,
            'mineral_probabilities': probabilities,
            'rock_analysis': rock_analysis
        }
    
    def plot_analysis(self, rock_num, save_path: str = None):
        """
        Create comprehensive visualization of the analysis results
        """
        if not self.analysis_history:
            print("No data to plot")
            return
            
        plt.style.use('ggplot')
        # fig = plt.figure(figsize=(15, 12))
        fig = plt.figure(figsize=(10, 6), dpi=300)
        gs = plt.GridSpec(3, 1, height_ratios=[2, 1, 1])
        
        # Plot 1: Mineral Predictions vs Ground Truth
        ax1 = fig.add_subplot(gs[0])
        measurements = range(1, len(self.analysis_history) + 1)
        ax1.set_xticks(measurements)
        
        # Plot ground truth
        true_minerals = [a['true_mineral'] for a in self.analysis_history]
        predicted_minerals = [a['mineral_prediction'] for a in self.analysis_history]

        print("ground-truth",true_minerals)
        print("predictions",predicted_minerals)
        
        ax1.scatter(measurements, true_minerals, label='Ground Truth', 
                marker='o', s=100, alpha=0.6)
        ax1.scatter(measurements, predicted_minerals, label='Predicted',
                marker='x', s=100, alpha=0.8)
        
        ax1.set_title(f'Mineral Predictions vs Ground Truth of Test Sample {rock_num}', fontsize=14, fontweight='bold', pad=20)
        ax1.set_xlabel('Measurement Number', fontsize=12, fontweight='bold')
        ax1.set_ylabel('Mineral', fontsize=12, fontweight='bold')
        ax1.legend(loc='upper left')
        plt.setp(ax1.get_xticklabels())

        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        else:
            plt.show()
        
        plt.close()
        
        # # Plot 2: Classification Results per Window
        # ax2 = fig.add_subplot(gs[1])
        # window_results = []
        # window_start = 0
        
        # for i in range(len(self.analysis_history)):
        #     if i >= 9:  # We have a full window of 10 measurements
        #         window_true = true_minerals[window_start:i+1]
        #         window_pred = predicted_minerals[window_start:i+1]
                
        #         # Calculate accuracy
        #         correct_count = sum(1 for t, p in zip(window_true, window_pred) if t == p)
        #         accuracy = correct_count / 10
                
        #         # Check mineral assemblage
        #         assemblage = self.check_mineral_assemblage_rules(window_pred)
                
        #         # Store window results
        #         window_results.append({
        #             'window': f'{window_start+1}-{i+1}',
        #             'accuracy': accuracy,
        #             'accuracy_satisfied': accuracy >= 0.6,
        #             'assemblage_satisfied': assemblage['satisfied'],
        #             'is_granite': accuracy >= 0.6 and assemblage['satisfied']
        #         })
        #         window_start += 1
        
        # if window_results:
        #     # Create classification result visualization
        #     windows = [w['window'] for w in window_results]
        #     x = np.arange(len(windows))
        #     bar_width = 0.35
            
        #     # Plot accuracy bars
        #     accuracy_bars = ax2.bar(x - bar_width/2, 
        #                         [w['accuracy'] for w in window_results],
        #                         bar_width, label='Accuracy', color='lightblue')
            
        #     # Add accuracy threshold line
        #     ax2.axhline(y=0.6, color='red', linestyle='--', label='Required Accuracy (60%)')
            
        #     # Add granite classification markers
        #     for i, result in enumerate(window_results):
        #         if result['is_granite']:
        #             ax2.text(i, 1.05, '✓ Granite', ha='center', va='bottom', color='green')
        #         else:
        #             ax2.text(i, 1.05, '✗ Not Granite', ha='center', va='bottom', color='red')
            
        #     # Add accuracy labels
        #     for i, rect in enumerate(accuracy_bars):
        #         height = rect.get_height()
        #         ax2.text(rect.get_x() + rect.get_width()/2., height,
        #                 f'{height:.0%}',
        #                 ha='center', va='bottom')
            
        #     ax2.set_title('Classification Results per Window')
        #     ax2.set_xlabel('Measurement Windows')
        #     ax2.set_ylabel('Accuracy')
        #     ax2.set_xticks(x)
        #     ax2.set_xticklabels(windows, rotation=45)
        #     ax2.set_ylim(0, 1.2)  # Make room for granite labels
        #     ax2.legend()
        
        # # Plot 3: Mineral Assemblage Analysis (Bar Chart)
        # ax3 = fig.add_subplot(gs[2])
        
        # # Get mineral counts for each window
        # window_data = []
        # window_start = 0
        
        # for i in range(len(self.analysis_history)):
        #     if i >= 9:  # We have a full window
        #         window_pred = predicted_minerals[window_start:i+1]
        #         assemblage = self.check_mineral_assemblage_rules(window_pred)
        #         counts = assemblage['counts']
        #         window_data.append({
        #             'window': f'{window_start+1}-{i+1}',
        #             'counts': counts
        #         })
        #         window_start += 1
        
        # if window_data:
        #     # Prepare data for stacked bar chart
        #     windows = [d['window'] for d in window_data]
        #     feldspar_counts = [d['counts']['feldspars'] for d in window_data]
        #     quartz_counts = [d['counts']['quartz'] for d in window_data]
        #     mica_counts = [d['counts']['micas'] for d in window_data]
            
        #     # Create stacked bar chart
        #     bar_width = 0.8
            
        #     # Plot bars with different colors
        #     bars1 = ax3.bar(range(len(windows)), feldspar_counts, bar_width,
        #                 label='Feldspars', color='#1f77b4', alpha=0.7)
        #     bars2 = ax3.bar(range(len(windows)), quartz_counts, bar_width,
        #                 bottom=feldspar_counts, label='Quartz', color='#2ca02c', alpha=0.7)
            
        #     # Calculate the total height for the mica bars
        #     total_height = [f + q for f, q in zip(feldspar_counts, quartz_counts)]
        #     bars3 = ax3.bar(range(len(windows)), mica_counts, bar_width,
        #                 bottom=total_height, label='Micas', color='#ff7f0e', alpha=0.7)
            
        #     # Customize the plot
        #     ax3.set_title('Mineral Group Counts per Window')
        #     ax3.set_xlabel('Measurement Windows')
        #     ax3.set_ylabel('Count')
        #     ax3.set_xticks(range(len(windows)))
        #     ax3.set_xticklabels(windows, rotation=45)
        #     ax3.legend()
            
        #     # Add count labels on the bars
        #     def add_labels(bars, bottom=None):
        #         for bar in bars:
        #             height = bar.get_height()
        #             if height > 0:  # Only add label if there's a count
        #                 y_pos = bar.get_y() + height/2.
        #                 ax3.text(bar.get_x() + bar.get_width()/2., y_pos,
        #                         f'{int(height)}',
        #                         ha='center', va='center')
            
        #     add_labels(bars1)
        #     add_labels(bars2)
        #     add_labels(bars3)
            
        #     # Add horizontal line for minimum requirements
        #     ax3.axhline(y=1, color='r', linestyle='--', alpha=0.5, 
        #                 label='Minimum Required (1)')
            
        # plt.tight_layout()
        
        # if save_path:
        #     plt.savefig(save_path, dpi=300, bbox_inches='tight')
        # else:
        #     plt.show()
        
        # plt.close()


In [4]:
def save_analysis(result, filename="rock_analysis_results.txt"):
    """
    Save rock analysis results to a text file
    
    Args:
        result (dict): Analysis result dictionary
        filename (str): Name of output file
    """
    with open(filename, 'w') as f:
        if 'accuracy_rule' in result['rock_analysis']:
            f.write(f"\nAnalysis Results:\n")
            f.write(f"Classification: {result['rock_analysis']['classification']}\n")
            f.write(f"Accuracy: {result['rock_analysis']['accuracy_rule']['accuracy']:.1%}\n")
            f.write("\nMineral Assemblage:\n")
            for rule, satisfied in result['rock_analysis']['assemblage_rules']['details'].items():
                f.write(f"- {rule}: {'✓' if satisfied else '✗'}\n")

In [7]:


def main():
    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model configuration and loading
    best_repeat = 0
    best_fold = 0
    best_overall_accuracy = 0.9927
    best_model_path = f'./models/model_repeat{best_repeat}_fold{best_fold}_acc{best_overall_accuracy:.4f}.pth'
    
    # Initialize model and load weights
    best_model = SimpleCNN1D().to(device)  # Make sure to import your model class
    best_checkpoint = torch.load(best_model_path)
    best_model.load_state_dict(best_checkpoint['model_state_dict'])
    best_model.eval()
    
    # Load label encoder
    label_encoder = joblib.load('mineral_label_encoder.joblib')
    
    # Initialize rock analyzer
    rock_analyzer = IntegratedRockClassifier(
        model=best_model,
        label_encoder=label_encoder,
        device=device,
    )
    
    # Load and process data
    # rock_num = 1
    base_path = Path("/home/iyeszin/Desktop/my git/minerals-rocks-relationship/all about rruff/validation_rocks")
    results = []
    
    for rock_num in range(1, 31):
        rock_analyzer.analysis_history = []  # Reset analysis history

        # Process rock
        final_classification = None
        final_accuracy = None

        print(f"\nProcessing Rock {rock_num}")
        rock_folder = f"rock_{rock_num:02d}"  # This will give "rock_01", "rock_02", etc.
        rock_path = base_path / rock_folder
        wavelengths = np.load(f"{rock_path}_wavelengths.npy")
        intensities = np.load(f"{rock_path}_intensities.npy")
        
        # Load true composition
        true_compositions = []
        composition_file = rock_path.with_name(f"{rock_path.stem}_composition.txt")
        with open(composition_file, 'r') as f:
            next(f)  # Skip header
            true_compositions = [line.strip().split('\t')[1] for line in f]
        
        # Process spectra and collect results
        for i, spectrum in enumerate(intensities):
            # Prepare spectrum tensor
            spectrum_tensor = torch.from_numpy(spectrum).float()
            
            # Process spectrum
            result = rock_analyzer.process_spectrum(spectrum_tensor, true_compositions[i])
            
            # Print analysis if we have a full window
            if 'accuracy_rule' in result['rock_analysis']:
                final_classification = result['rock_analysis']['classification']
                final_accuracy = result['rock_analysis']['accuracy_rule']['accuracy']

                print(f"\nAnalysis for measurements 1-10:")
                print(f"Classification: {result['rock_analysis']['classification']}")
                print(f"Accuracy: {result['rock_analysis']['accuracy_rule']['accuracy']:.1%}")
                print("Mineral Assemblage:")
                for rule, satisfied in result['rock_analysis']['assemblage_rules']['details'].items():
                    print(f"- {rule}: {'✓' if satisfied else '✗'}")
            
        # Append only once per rock
        if final_classification and final_accuracy:
            results.append({
                'rock_num': rock_num,
                'classification': final_classification, 
                'accuracy': final_accuracy
            })
                
        
        save_analysis(result, 'rock'+str(rock_num)+'_analysis_results.txt')  # Saves to 'rock_analysis_results.txt'


        # Create final visualization
        rock_analyzer.plot_analysis(rock_num, 'rock_analysis'+str(rock_num)+'.png')

    # Save results after each rock
    pd.DataFrame(results).to_csv('rock_classifications.csv', index=False)

if __name__ == "__main__":
    main()

/tmp/ipykernel_189518/2464718170.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_checkpoint = torch.load(best_model_path)



Processing Rock 1

Analysis for measurements 1-10:
Classification: granite
Accuracy: 80.0%
Mineral Assemblage:
- granite: ✓
- limestone: ✓
- sandstone: ✓
ground-truth ['Albite', 'Anorthite', 'Quartz', 'Quartz', 'Annite', 'Muscovite', 'Quartz', 'Albite', 'Annite', 'Orthoclase']
predictions ['Albite', 'Anorthite', 'Quartz', 'Quartz', 'Annite', 'Muscovite', 'Quartz', 'Anorthite', 'Annite', 'Sanidine']
  agg_filter: a filter function, which takes a (m, n, 3) float array and a dpi value, and returns a (m, n, 3) array and two offsets from the bottom left corner of the image
  alpha: scalar or None
  animated: bool
  antialiased: bool
  backgroundcolor: color
  bbox: dict with properties for `.patches.FancyBboxPatch`
  clip_box: `~matplotlib.transforms.BboxBase` or None
  clip_on: bool
  clip_path: Patch or (Path, Transform) or None
  color or c: color
  figure: `~matplotlib.figure.Figure`
  fontfamily or family or fontname: {FONTNAME, 'serif', 'sans-serif', 'cursive', 'fantasy', 'monospace'

In [8]:
def analyze_results():
    import pandas as pd
    from sklearn.metrics import confusion_matrix, classification_report
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    # Load saved classifications
    results_df = pd.read_csv('rock_classifications.csv')
    
    # Ground truth list
    # Borderline is consider the rock type
    # Not the rock type is consider other
    ground_truths = [
        'granite', 'granite', 'granite', 'granite', 'granite', 
        'granite', 'granite', 'granite', 'other', 'other',
        'sandstone', 'sandstone', 'sandstone', 'sandstone', 'sandstone',
        'sandstone', 'other', 'sandstone', 'sandstone', 'sandstone',
        'limestone', 'other', 'limestone', 'other', 'limestone',
        'limestone', 'limestone', 'limestone', 'limestone', 'limestone'
    ]
    
    # Create ground truth dataframe
    ground_truth_df = pd.DataFrame({
        'rock_num': range(1, 31),
        'ground_truth': ground_truths
    })
    
    # Merge and analyze
    final_df = pd.merge(results_df, ground_truth_df, on='rock_num')
    
    cm = confusion_matrix(final_df['ground_truth'], final_df['classification'])
    labels = ['Granite', 'Limestone', 'Sandstone', 'Other']
    print("\nConfusion Matrix:")
    print(cm)

    # Plot confusion matrix
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, 
            yticklabels=labels)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True Label')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    print("\nClassification Report:")
    print(classification_report(final_df['ground_truth'], final_df['classification']))


    final_df.to_csv('rock_classification_analysis.csv', index=False)

if __name__ == "__main__":
    analyze_results()


Confusion Matrix:
[[8 0 0 0]
 [0 6 2 0]
 [3 1 1 0]
 [9 0 0 0]]

Classification Report:
              precision    recall  f1-score   support

     granite       0.40      1.00      0.57         8
   limestone       0.86      0.75      0.80         8
       other       0.33      0.20      0.25         5
   sandstone       0.00      0.00      0.00         9

    accuracy                           0.50        30
   macro avg       0.40      0.49      0.41        30
weighted avg       0.39      0.50      0.41        30



/home/iyeszin/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iyeszin/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/iyeszin/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
